Mounting Google Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Copying Dataset from Google Drive to Colab

In [ ]:
!pwd

In [ ]:
!cp /content/drive/MyDrive/vehicle_detection_dataset.zip /content

Unzipping the Dataset

In [ ]:
!unzip /content/vehicle_detection_dataset.zip -d /content

#Exploring the Dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
for img in os.listdir('/content/vehicle_detection_dataset/train/images')[:10]:
  path = f'/content/vehicle_detection_dataset/train/images/{img}'
  img_read = plt.imread(path)
  plt.imshow(img_read)
  plt.show()

Drawing the Bounding Box

In [ ]:
#read the txt file(read single file only)
def parse_yolo_annotation(annotation_path):
  with open(annotation_path, 'r') as file:
    lines = file.read().split('\n')
    annotations = []
    for line in lines:
      if line.strip() != "":
        data = line.split()
        class_id = int(data[0])
        x,y,w,h = map(float, data[1:])
        var_dict = {'class_id' : class_id, 'x' : x, 'y' : y, 'w' : w, 'h' : h}
        annotations.append(var_dict)
  return annotations


In [ ]:
data_dir = '/content/vehicle_detection_dataset/valid'
image_files = [f for f in os.listdir(f'{data_dir}/images')[:10] if f.endswith('.jpg')]
for image_file in image_files:
  img_path = f'/content/vehicle_detection_dataset/valid/images/{image_file}'
  image = plt.imread(img_path)
  #changing the path to txt
  annotation_path = f'/content/vehicle_detection_dataset/valid/labels/{image_file[:-4]}.txt'
  annotations = parse_yolo_annotation(annotation_path)

  class_color_dict = {
    0: (255, 0, 0),      # Ambulance - Red
    1: (0, 255, 0),      # Auto - Green
    2: (0, 0, 255),      # Bicycle - Blue
    3: (255, 255, 0),    # Bike - Yellow
    4: (255, 165, 0),    # Bus - Orange
    5: (0, 255, 255),    # Car - Cyan
    6: (128, 0, 128),    # Motorcycle - Purple
    7: (128, 128, 128),  # Truck - Gray
    8: (255, 255, 255),  # auto-rickshaw - White
    9: (0, 128, 0),      # bus - Dark Green
    10: (0, 0, 128),     # car - Dark Blue
    11: (128, 0, 0),     # lorry - Dark Red
    12: (128, 128, 0),   # mini truck - Olive
    13: (255, 0, 255),   # small-motorcycle - Magenta
    14: (128, 64, 0)     # smalltruck - Brown
    }

  for annotation in annotations:
    print(annotation)
    class_id = annotation['class_id']
    x = annotation['x']
    y = annotation['y']
    w = annotation['w']
    h = annotation['h']

    color = class_color_dict.get(class_id, (0,0,0))
    img_h,img_w,channel = image.shape
    x1 = int((x - w/2) * img_w)
    y1 = int((y - h/2) * img_h)

    x2 = int((x + w/2) * img_w)
    y2 = int((y + h/2) * img_h)

    cv2.rectangle(image,(x1, y1),(x2, y2), color, 10)

  plt.imshow(image)
  plt.show()

#*Training the model(“vehicle_data.yaml” file can also be used by configuring the path)*

In [ ]:
import yaml

In [ ]:
data_yaml = dict(
    train = '/content/vehicle_detection_dataset/train',
    val = '/content/vehicle_detection_dataset/valid',
    nc = 15,
    names = ['Ambulance', 'Auto', 'Bicycle', 'Bike', 'Bus', 'Car', 'Motorcycle', 'Truck', 'auto-rickshaw', 'bus', 'car', 'lorry', 'mini truck', 'motorcycle', 'truck']
)


with open('vehicle_name.yaml','w') as file:
  yaml.dump(data_yaml, file)

Installing Ultralytics

In [ ]:
!pip install ultralytics

In [ ]:
!yolo detect train data=/content/vehicle_name.yaml model=yolov8s.pt epochs=10 imgsz=640

#Inference the Yolov8

In [ ]:
!yolo predict model = /content/runs/detect/train/weights/best.pt imgsz=640 conf=0.25 source = /content/vehicle_detection_dataset/valid/images

In [ ]:
!cp -r runs /content/drive/MyDrive/Detect_Vehicle_Yolov8